In [1]:
from ctypes import (
    cdll,
    c_int,
    c_double,
    c_longlong,
    c_voidp,
    POINTER,
)
import pickle
import numpy as np


In [9]:
spacc = cdll.LoadLibrary("spacc.so")

spacc.test.restype = c_int

spacc.spacc_create_sparse_matrix.restype = c_int
spacc.spacc_create_sparse_matrix.argtypes = [
    c_int,
    c_int,
    c_int,
    c_int,
    POINTER(c_longlong),
    POINTER(c_longlong),
    POINTER(c_double),
]
spacc.spacc_gemv.restype = c_int
spacc.spacc_gemv.argtypes = [c_double, c_int, POINTER(c_double), POINTER(c_double)]

spacc.spacc_daxpy.restype = c_voidp
spacc.spacc_daxpy.argtypes = [c_int, c_double, POINTER(c_double), POINTER(c_double)]


In [8]:
spacc.test()


0

Overwriting existing matrix @ store_idx 0.
Matrix added at 0
Res: 201.000000
Res: 310.000000
Res: 104.000000
Matrix destroyed at 0


In [4]:
from scipy.sparse import coo_matrix

coom = coo_matrix(np.random.random((3, 3)))


In [5]:
class AccelerateSparseMatrix(object):
    def __init__(self, scipy_sparse_matrix):
        spm = scipy_sparse_matrix.tocoo()
        self.dim_rows, self.dim_cols = spm.shape
        self.nnz = spm.nnz
        self.col = spm.col.astype("longlong")
        self.row = spm.row.astype("longlong")
        self.data = spm.data.astype("double")
        self.store_id = None
        self._init_spacc_matrix()

    def _init_spacc_matrix(self):
        self.store_id = spacc.spacc_create_sparse_matrix(
            -1,
            self.dim_rows,
            self.dim_cols,
            self.nnz,
            self.row.ctypes.data_as(POINTER(c_longlong)),
            self.col.ctypes.data_as(POINTER(c_longlong)),
            self.data.ctypes.data_as(POINTER(c_double)),
        )
        if self.store_id < 0:
            raise Exception("Matrix creation failed.")

    def gemv(self, alpha, x, y):
        assert x.shape[0] == self.dim_cols
        assert y.shape[0] == self.dim_cols
        assert x.dtype == "float64"
        assert y.dtype == "float64"
        alpha = np.double(alpha)

        cy = y.ctypes.data_as(POINTER(c_double))
        cx = x.ctypes.data_as(POINTER(c_double))
        if spacc.spacc_gemv(alpha, self.store_id, cx, cy) != 0:
            raise Exception("Sparse matrix-vector multiplication failed.")


In [6]:
int_m = pickle.load(open("int_m.pkl", "rb"))


FileNotFoundError: [Errno 2] No such file or directory: 'int_m.pkl'

In [ ]:
aspm = AccelerateSparseMatrix(int_m)


Assigned free store_idx 0.
Matrix added at 0


In [12]:
y = np.zeros(10000).astype("double")
x = np.random.random(10000).astype("double")


In [ ]:
%timeit aspm.gemv(1.,x,y)

910 µs ± 22.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%timeit int_m.dot(x)

4.4 ms ± 75.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
spacc.spacc_daxpy(len(x), 1., cx, cy)


NameError: name 'cx' is not defined

In [ ]:
y


array([-4.34851174e-01,  2.21951044e-01,  2.97501411e-02, ...,
        5.00273757e-10,  3.67755744e-12,  0.00000000e+00])

In [ ]:
npy = int_m.dot(x)
